In [ ]:
#13-11-2023 
#Shadow removal trial

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam

def load_shadow_dataset(dataset_path, target_size=(256, 256)):
    shadow_images = []
    shadow_free_images = []

    for filename in os.listdir(dataset_path):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(dataset_path, filename)
            print(f"Processing image: {image_path}")

            image = cv2.imread(image_path)

            # Resize the image to a common size
            image = cv2.resize(image, target_size)
            # Normalize pixel values to be in the range [0, 1]
            image = image / 255.0

            # Simulate having shadow and shadow-free pairs
            shadow_images.append(image)

            # For simplicity, assume shadow-free images have "_sf" in their filenames
            shadow_free_filename = filename.replace(os.path.splitext(filename)[1], "_sf" + os.path.splitext(filename)[1])
            shadow_free_image_path = os.path.join(dataset_path, shadow_free_filename)

            # Use a raw string for the path
            shadow_free_image_path = r'' + shadow_free_image_path
            print(f"Expected shadow-free image path: {shadow_free_image_path}")

            shadow_free_image = cv2.imread(image_path)

            # Check if the image is successfully loaded
            if shadow_free_image is not None:
                # Resize the shadow-free image to a common size
                shadow_free_image = cv2.resize(shadow_free_image, target_size)
                # Normalize pixel values to be in the range [0, 1]
                shadow_free_image = shadow_free_image / 255.0

                shadow_free_images.append(shadow_free_image)
            else:
                print(f"Failed to load {shadow_free_image_path}")

    return np.array(shadow_images), np.array(shadow_free_images)


def build_shadow_removal_model(input_shape):
    model = Sequential()

    # Encoder
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2), padding='same'))

    # Decoder
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    return model

# Replace 'path_to_your_dataset' with the actual path to your dataset
dataset_path = r'C:\\Users\\NARENDRA\\Desktop\\Cotton_Leaf_Disease_Detection\\Dataset1\\cotton\\bacterial_blight'


# Load the shadow dataset
shadow_images, shadow_free_images = load_shadow_dataset(dataset_path)


# Check if the dataset is loaded successfully
if len(shadow_images) == 0 or len(shadow_free_images) == 0:
    raise ValueError("Failed to load the shadow dataset. Please check the dataset path.")

# Build the shadow removal model
input_shape = shadow_images[0].shape
shadow_removal_model = build_shadow_removal_model(input_shape)

# Train the model
history = shadow_removal_model.fit(shadow_images, shadow_free_images, epochs=10, batch_size=32, validation_split=0.2)

# Plot training history
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Now, you can use the trained model to remove shadows from new images
# shadow_free_image = shadow_removal_model.predict(new_image)
# Assuming 'new_image_path' is the path to the new image for shadow removal
new_image_path = 'C:\\Users\\NARENDRA\\Desktop\\Cotton_Leaf_Disease_Detection\\Dataset1\\cotton\\bacterial_blight\\bact219.jpg'

# Load the new image
new_image = cv2.imread(new_image_path)

# Preprocess the new image (resize to match the model's input size and normalize pixel values)
new_image = cv2.resize(new_image, (input_shape[1], input_shape[0]))
new_image = new_image / 255.0

# Add a batch dimension to the input image
new_image = np.expand_dims(new_image, axis=0)

# Predict shadow-free image using the trained model
shadow_free_image = shadow_removal_model.predict(new_image)

# Reshape the output image and remove the batch dimension
shadow_free_image = np.squeeze(shadow_free_image)

# Display the original and shadow-free images
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title('Original Image')
# Convert the image to 8-bit unsigned integer format
original_image = cv2.convertScaleAbs(new_image[0])
plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Shadow-Free Image')
plt.imshow(cv2.cvtColor(shadow_free_image, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.show()
